In [ ]:
import requests
import json
import matplotlib.pyplot as plt
from PIL import Image
from io import BytesIO
from osgeo import gdal

In [ ]:
TOKEN = "Tellus のマイページ上で取得した API トークン "
# Tellus Traveler で探したデータの id を指定
dataset_id = "50f05191-a0d5-4377-be8d-a880ec54ae27"
scene_id = "5b552df4-9ee7-4387-b1a6-f8874dc6ba86"
files_url = f"https://www.tellusxdp.com/api/traveler/v1/datasets/{dataset_id}/data/{scene_id}/files/"
# Tellus の API は Bearer 認証のため、リクエストヘッダに必要な情報を設定
headers = {
    "Authorization": "Bearer " + TOKEN,
    "Content-Type": "application/json",
}
# headers で設定したヘッダ情報を用いて GET リクエストを送信し、結果を受け取る
response = requests.get(files_url, headers=headers)
json.loads(response.content)


In [ ]:
file_id = 11
api_download_url = f"https://www.tellusxdp.com/api/traveler/v1/datasets/{dataset_id}/data/{scene_id}/files/{file_id}/download-url/"
# 画像ファイルのダウンロード URL を生成する POST リクエストを送信し、生成された URL を受け取る
response = requests.post(api_download_url, headers=headers)
tif_download_url = json.loads(response.content)["download_url"]

In [ ]:
# stream オプションを有効にし、画像ファイルを chunk に分割したデータを 1 つのファイルに書き込む
tif_file_path = "./ASNARO.tif"
with requests.get(tif_download_url, stream=True) as r:
    r.raise_for_status()
    with open(tif_file_path, "wb") as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)

In [ ]:
src = gdal.Open(tif_file_path)

In [ ]:
b2 = src.GetRasterBand(2).ReadAsArray()

In [ ]:
# Matplotlib の pyplot.hist 関数を用いてヒストグラムを表示
plt.hist(b2.flatten(), bins=100, range=(1, b2.max()))
plt.title("BlueBand", fontsize=10)

In [ ]:
plt.imshow(b2, vmax=1200, cmap="Blues")

In [ ]:
gdal.Translate(
    "./tellus_travel_test.png",
    src,
    srcWin=(3500, 3400, 1280, 1280),
    scaleParams=[[0, 1200]],
    format="PNG",
    outputType=gdal.GDT_Byte,
    bandList=[4, 3, 2],
)

In [ ]:
image = Image.open("./tellus_travel_test.png")
image.show()

In [ ]:
# 前回作成した学習済みモデルデータ
MODEL_PATH = "./YOLOX_outputs/airbus_config/best_ckpt.pth"
# 画像サイズをモデルの学習時に利用していた大きさに揃える
resized_test_image = image.resize((640, 640))
resized_test_image.save("resized_test.jpg")

In [ ]:
!python ./YOLOX/tools/demo.py image ¥
    -f airbus̲config.py ¥ # 設定ファイルを指定
    -c {MODEL̲PATH} ¥ # 学習済みデータモデルのファイル指定
    --path ./resized̲test.jpg ¥ # 画像のファイルパス指定
    --tsize 640 ¥ # 対象画像の画像解像度を指定
    --save̲result ¥ # 結果画像を保存する
    --device gpu # 利用デバイスの指定（ここでは GPU か CPU か）